<a href="https://colab.research.google.com/github/rajawatvishal007/AQI-Prediction/blob/main/AQI_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive & Import Libraries

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load All City Datasets

In [2]:
import pandas as pd
Delhi = pd.read_csv('/content/drive/MyDrive/Delhi_data.csv', low_memory=False)
Gurugram = pd.read_csv('/content/drive/MyDrive/Gurugram_data.csv', low_memory=False)
Guwahati = pd.read_csv('/content/drive/MyDrive/Guwahati_data.csv', low_memory=False)
Mumbai = pd.read_csv('/content/drive/MyDrive/Mumbai_data.csv', low_memory=False)
Kolkata = pd.read_csv('/content/drive/MyDrive/Kolkata_data.csv', low_memory=False)
Bophal = pd.read_csv('/content/drive/MyDrive/Bhopal_data.csv', low_memory=False)
Jaipur = pd.read_csv('/content/drive/MyDrive/Jaipur_data.csv', low_memory=False)
shillong = pd.read_csv('/content/drive/MyDrive/Shillong_data.csv', low_memory=False)
Thiruvananthapuram = pd.read_csv('/content/drive/MyDrive/Thiruvananthapuram_data.csv', low_memory=False)
Amaravati = pd.read_csv('/content/drive/MyDrive/Amaravati_data.csv', low_memory=False)

In [3]:
dataframes = {
    'Delhi': Delhi,
    'Gurugram': Gurugram,
    'Guwahati': Guwahati,
    'Mumbai': Mumbai,
    'Kolkata': Kolkata,
    'Bophal': Bophal,
    'Jaipur': Jaipur,
    'Shillong': shillong,
    'Thiruvananthapuram': Thiruvananthapuram,
    'Amaravati': Amaravati
}

for name, df in dataframes.items():
    print(f"Shape of {name}: {df.shape}")

Shape of Delhi: (2009, 16)
Shape of Gurugram: (1679, 16)
Shape of Guwahati: (502, 16)
Shape of Mumbai: (2009, 16)
Shape of Kolkata: (814, 16)
Shape of Bophal: (289, 16)
Shape of Jaipur: (1114, 16)
Shape of Shillong: (310, 16)
Shape of Thiruvananthapuram: (1112, 16)
Shape of Amaravati: (951, 16)


Display Shapes & Initial Checks

In [4]:
print(Delhi.isnull().sum())

City            0
Date            0
PM2.5           2
PM10           77
NO              2
NO2             2
NOx             0
NH3             9
CO              0
SO2           110
O3             84
Benzene         0
Toluene         0
Xylene        781
AQI            10
AQI_Bucket     10
dtype: int64


In [5]:
Delhi.shape

(2009, 16)

Combine All City Data

In [6]:
all_cities = pd.concat(dataframes.values(), ignore_index=True)
print(f"Shape of all_cities: {all_cities.shape}")
print("First 5 rows of all_cities:")
print(all_cities.head())

Shape of all_cities: (10789, 16)
First 5 rows of all_cities:
    City        Date   PM2.5    PM10     NO    NO2     NOx     NH3     CO  \
0  Delhi  01/01/2015  313.22  607.98  69.16  36.39  110.59   33.85  15.20   
1  Delhi  02/01/2015  186.18  269.55  62.09  32.87   88.14   31.83   9.54   
2  Delhi  03/01/2015   87.18  131.90  25.73  30.31   47.95   69.55  10.61   
3  Delhi  04/01/2015  151.84  241.84  25.01  36.91   48.62  130.36  11.54   
4  Delhi  05/01/2015  146.60  219.13  14.01  34.92   38.25  122.88   9.20   

    SO2     O3  Benzene  Toluene  Xylene    AQI AQI_Bucket  
0  9.25  41.68    14.36    24.86    9.84  472.0     Severe  
1  6.65  29.97    10.55    20.09    4.29  454.0     Severe  
2  2.65  19.71     3.91    10.23    1.99  143.0   Moderate  
3  4.63  25.36     4.26     9.71    3.34  319.0  Very Poor  
4  3.33  23.20     2.80     6.21    2.96  325.0  Very Poor  


Clean Data (Missing Values + Imputation)

In [7]:
import numpy as np
# Drop columns with 100% missing values in all_cities
cols_to_drop_100_percent_missing_all_cities = [col for col in all_cities.columns if all_cities[col].isnull().all()]
if cols_to_drop_100_percent_missing_all_cities:
    print(f"Dropping 100% missing columns in all_cities: {cols_to_drop_100_percent_missing_all_cities}")
    all_cities = all_cities.drop(columns=cols_to_drop_100_percent_missing_all_cities)

# Impute numerical columns with median in all_cities
numerical_cols_all_cities = all_cities.select_dtypes(include=np.number).columns
for col in numerical_cols_all_cities:
    if all_cities[col].isnull().any():
        median_val = all_cities[col].median()
        all_cities[col] = all_cities[col].fillna(median_val)
        print(f"  - Imputed numerical column {col} in all_cities with median: {median_val}")

# Impute categorical column (AQI_Bucket) with mode in all_cities
if 'AQI_Bucket' in all_cities.columns and all_cities['AQI_Bucket'].isnull().any():
    mode_val = all_cities['AQI_Bucket'].mode()[0]
    all_cities['AQI_Bucket'] = all_cities['AQI_Bucket'].fillna(mode_val)
    print(f"  - Imputed categorical column AQI_Bucket in all_cities with mode: {mode_val}")

# Verify no more missing values in all_cities
if all_cities.isnull().sum().sum() == 0:
    print(f"\nNo missing values remaining in all_cities after cleaning.")
else:
    print(f"\nMissing values still present in all_cities after cleaning:")
    print(all_cities.isnull().sum()[all_cities.isnull().sum() > 0])

  - Imputed numerical column PM2.5 in all_cities with median: 51.3
  - Imputed numerical column PM10 in all_cities with median: 104.33
  - Imputed numerical column NO in all_cities with median: 10.4
  - Imputed numerical column NO2 in all_cities with median: 23.22
  - Imputed numerical column NOx in all_cities with median: 28.2
  - Imputed numerical column NH3 in all_cities with median: 18.18
  - Imputed numerical column CO in all_cities with median: 0.78
  - Imputed numerical column SO2 in all_cities with median: 10.535
  - Imputed numerical column O3 in all_cities with median: 35.805
  - Imputed numerical column Benzene in all_cities with median: 1.22
  - Imputed numerical column Toluene in all_cities with median: 4.46
  - Imputed numerical column Xylene in all_cities with median: 0.14
  - Imputed numerical column AQI in all_cities with median: 122.0
  - Imputed categorical column AQI_Bucket in all_cities with mode: Satisfactory

No missing values remaining in all_cities after cleani

In [8]:
all_cities.shape

(10789, 16)

In [9]:
all_cities

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Delhi,01/01/2015,313.22,607.98,69.16,36.39,110.59,33.85,15.20,9.25,41.68,14.36,24.86,9.84,472.0,Severe
1,Delhi,02/01/2015,186.18,269.55,62.09,32.87,88.14,31.83,9.54,6.65,29.97,10.55,20.09,4.29,454.0,Severe
2,Delhi,03/01/2015,87.18,131.90,25.73,30.31,47.95,69.55,10.61,2.65,19.71,3.91,10.23,1.99,143.0,Moderate
3,Delhi,04/01/2015,151.84,241.84,25.01,36.91,48.62,130.36,11.54,4.63,25.36,4.26,9.71,3.34,319.0,Very Poor
4,Delhi,05/01/2015,146.60,219.13,14.01,34.92,38.25,122.88,9.20,3.33,23.20,2.80,6.21,2.96,325.0,Very Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10784,Amaravati,27/06/2020,14.50,24.43,1.53,6.53,4.72,8.97,0.55,13.75,33.76,0.06,0.27,0.15,42.0,Good
10785,Amaravati,28/06/2020,16.65,28.51,1.43,8.32,5.59,9.77,0.66,10.86,37.34,0.10,0.43,0.12,49.0,Good
10786,Amaravati,29/06/2020,20.96,32.56,1.65,9.55,6.43,14.30,0.66,14.79,43.29,0.12,0.69,0.10,56.0,Satisfactory
10787,Amaravati,30/06/2020,21.34,35.16,1.74,10.69,7.10,13.38,0.66,14.58,45.32,0.14,1.42,0.20,61.0,Satisfactory


Exploratory Data Analysis (EDA)

Value Counts

In [10]:
all_cities['City'].value_counts()

,count
City,
Delhi,2009
Mumbai,2009
Gurugram,1679
Jaipur,1114
Thiruvananthapuram,1112
Amaravati,951
Kolkata,814
Guwahati,502
Shillong,310


In [11]:
all_cities['AQI_Bucket'].value_counts()

,count
AQI_Bucket,
Satisfactory,4728
Moderate,2720
Poor,1267
Very Poor,1016
Good,704
Severe,354


In [12]:
import plotly.express as px
avg_aqi_per_bucket = all_cities.groupby('AQI_Bucket')['AQI'].mean().reset_index()

# Define a specific order for AQI_Bucket for better visualization
aqi_bucket_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
avg_aqi_per_bucket['AQI_Bucket'] = pd.Categorical(avg_aqi_per_bucket['AQI_Bucket'], categories=aqi_bucket_order, ordered=True)
avg_aqi_per_bucket = avg_aqi_per_bucket.sort_values('AQI_Bucket')

fig = px.bar(avg_aqi_per_bucket,
             x='AQI_Bucket',
             y='AQI',
             color='AQI_Bucket',
             title='Average AQI per Air Quality Bucket',
             labels={'AQI': 'Average Air Quality Index', 'AQI_Bucket': 'Air Quality Bucket'},
             category_orders={'AQI_Bucket': aqi_bucket_order},
             color_discrete_map={
                 'Good': 'green',
                 'Satisfactory': 'lime',
                 'Moderate': 'yellow',
                 'Poor': 'orange',
                 'Very Poor': 'darkorange',
                 'Severe': 'red'
             },
             hover_data={'AQI_Bucket': True, 'AQI': ':.2f'})

fig.update_layout(xaxis_title='Air Quality Bucket', yaxis_title='Average AQI')
fig.show()

Average AQI per Bucket (Bar Chart)

AQI Bucket Distribution (Doughnut Chart)

In [13]:
aqi_bucket_counts = all_cities['AQI_Bucket'].value_counts().reset_index()
aqi_bucket_counts.columns = ['AQI_Bucket', 'Count']

# Define a specific order for AQI_Bucket for better visualization and consistent coloring
aqi_bucket_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
aqi_bucket_counts['AQI_Bucket'] = pd.Categorical(aqi_bucket_counts['AQI_Bucket'], categories=aqi_bucket_order, ordered=True)
aqi_bucket_counts = aqi_bucket_counts.sort_values('AQI_Bucket')

fig = px.pie(aqi_bucket_counts,
             values='Count',
             names='AQI_Bucket',
             title='Distribution of Air Quality Buckets',
             hole=0.4, # Creates the donut chart effect
             color='AQI_Bucket', # Color by AQI_Bucket
             color_discrete_map={
                 'Good': 'green',
                 'Satisfactory': 'lime',
                 'Moderate': 'yellow',
                 'Poor': 'orange',
                 'Very Poor': 'darkorange',
                 'Severe': 'red'
             },
             hover_data=['AQI_Bucket', 'Count'], # Corrected from dictionary to list
             labels={'AQI_Bucket': 'Air Quality Bucket', 'Count': 'Number of Records'})

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Time Series Plot of AQI by Date

In [14]:
# Convert 'Date' column to datetime objects
all_cities['Date'] = pd.to_datetime(all_cities['Date'], format='%d/%m/%Y', errors='coerce')

# Drop rows where 'Date' conversion failed (if any)
all_cities.dropna(subset=['Date'], inplace=True)

# Sort data by Date for proper time series plotting
all_cities.sort_values(by='Date', inplace=True)

# Create the line plot for AQI over time, colored by City
fig = px.line(all_cities,
              x='Date',
              y='AQI',
              color='City',
              title='Time Series Trend of AQI by City',
              labels={'AQI': 'Air Quality Index', 'Date': 'Date'},
              hover_data={'City': True, 'Date': True, 'AQI': True})

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Air Quality Index (AQI)",
    hovermode="x unified"
)

fig.show()

City-wise AQI Bucket Distribution

In [15]:
aqi_distribution = all_cities.groupby(['City', 'AQI_Bucket']).size().reset_index(name='Count')

fig = px.bar(aqi_distribution,
             x='City',
             y='Count',
             color='AQI_Bucket',
             title='City-wise Distribution of Air Quality Buckets',
             labels={'Count': 'Number of Records', 'City': 'City', 'AQI_Bucket': 'AQI Bucket'},
             category_orders={'AQI_Bucket': ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']},
             color_discrete_map={
                 'Good': 'green',
                 'Satisfactory': 'lime',
                 'Moderate': 'yellow',
                 'Poor': 'orange',
                 'Very Poor': 'darkorange',
                 'Severe': 'red'
             },
             hover_data={'City': True, 'AQI_Bucket': True, 'Count': True})

fig.update_layout(xaxis_title='City', yaxis_title='Number of Records')
fig.show()

Boxplots for Major Pollutants

In [16]:
pollutants = ['PM2.5', 'NO2', 'O3']

for pollutant in pollutants:
    fig = px.box(all_cities,
                 x='City',
                 y=pollutant,
                 color='City',
                 title=f'Distribution of {pollutant} Concentration by City',
                 labels={pollutant: f'{pollutant} Concentration', 'City': 'City'},
                 hover_data={'City': True, pollutant: True})
    fig.update_layout(xaxis_title='City', yaxis_title=f'{pollutant} Concentration')
    fig.show()

Average Pollutant Concentrations

In [17]:
pollutant_cols = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene']

average_pollutant_concentration = all_cities[pollutant_cols].mean().reset_index()
average_pollutant_concentration.columns = ['Pollutant', 'Average Concentration']

fig = px.bar(average_pollutant_concentration,
             x='Pollutant',
             y='Average Concentration',
             color='Pollutant', # Color each bar by its pollutant type
             title='Overall Average Pollutant Concentration (Bar Chart)',
             labels={'Pollutant': 'Pollutant Type', 'Average Concentration': 'Average Concentration'},
             hover_data={'Pollutant': True, 'Average Concentration': ':.2f'})

fig.update_layout(xaxis_title='Pollutant Type', yaxis_title='Average Concentration')
fig.show()

Violin Plot (Good vs Severe)

In [18]:
filtered_data = all_cities[all_cities['AQI_Bucket'].isin(['Good', 'Severe'])]

fig = px.violin(filtered_data,
                x='AQI_Bucket',
                y='PM2.5',
                color='AQI_Bucket',
                box=True, # Show box plot inside the violin
                points='all', # Show all data points
                title='PM2.5 Distribution in Good vs. Severe Air Quality',
                labels={'PM2.5': 'PM2.5 Concentration', 'AQI_Bucket': 'Air Quality Bucket'},
                category_orders={'AQI_Bucket': ['Good', 'Severe']},
                hover_data={'City': True, 'Date': True, 'AQI': True})

fig.update_layout(xaxis_title='Air Quality Bucket', yaxis_title='PM2.5 Concentration')
fig.show()

Correlation Heatmap

In [19]:
numerical_cols_all_cities = all_cities.select_dtypes(include=np.number)
correlation_matrix_all_cities = numerical_cols_all_cities.corr()

fig = px.imshow(correlation_matrix_all_cities,
                 text_auto=True,
                 aspect="auto",
                 title="Correlation Heatmap for All Cities (Numerical Pollutants)",
                 color_continuous_scale='Jet' # Changed to 'Jet' for more colorful scale
                )

fig.update_layout(xaxis_title="Pollutant", yaxis_title="Pollutant")
fig.show()

In [20]:
all_cities

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Delhi,2015-01-01,313.22,607.98,69.16,36.39,110.59,33.85,15.20,9.250,41.680,14.36,24.86,9.84,472.0,Severe
4190,Mumbai,2015-01-01,51.30,104.33,10.40,23.22,27.38,18.18,0.00,10.535,35.805,0.00,0.00,0.00,122.0,Satisfactory
4191,Mumbai,2015-01-02,51.30,104.33,10.40,23.22,26.20,18.18,0.00,10.535,35.805,0.00,0.00,0.00,122.0,Satisfactory
1,Delhi,2015-01-02,186.18,269.55,62.09,32.87,88.14,31.83,9.54,6.650,29.970,10.55,20.09,4.29,454.0,Severe
2,Delhi,2015-01-03,87.18,131.90,25.73,30.31,47.95,69.55,10.61,2.650,19.710,3.91,10.23,1.99,143.0,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6198,Mumbai,2020-07-01,3.84,11.28,27.64,5.57,33.21,23.60,0.37,13.180,6.680,0.78,4.46,0.14,53.0,Satisfactory
2008,Delhi,2020-07-01,54.01,128.66,6.33,21.05,16.81,29.06,0.97,11.150,29.730,2.03,23.57,0.65,101.0,Moderate
3687,Gurugram,2020-07-01,61.64,174.08,5.99,10.58,12.37,2.91,1.11,7.570,44.140,2.67,4.46,4.73,157.0,Moderate
7012,Kolkata,2020-07-01,10.53,31.58,6.25,10.01,16.29,12.87,0.30,4.320,15.520,1.82,10.25,0.18,45.0,Good


Label Encoding

In [21]:
from sklearn.preprocessing import LabelEncoder

# Create separate encoders
le_city = LabelEncoder()
le_bucket = LabelEncoder()

# Apply label encoding
all_cities['City_encoded'] = le_city.fit_transform(all_cities['City'])
all_cities['AQI_Bucket_encoded'] = le_bucket.fit_transform(all_cities['AQI_Bucket'])

# Show the mapping (important for interpretation)
print("City Encoding Mapping:")
for city, code in zip(le_city.classes_, le_city.transform(le_city.classes_)):
    print(f"{city} → {code}")

print("\n AQI_Bucket Encoding Mapping:")
for bucket, code in zip(le_bucket.classes_, le_bucket.transform(le_bucket.classes_)):
    print(f"{bucket} → {code}")

# Show encoded columns
all_cities[['City', 'City_encoded', 'AQI_Bucket', 'AQI_Bucket_encoded']].head()

City Encoding Mapping:
Amaravati → 0
Bhopal → 1
Delhi → 2
Gurugram → 3
Guwahati → 4
Jaipur → 5
Kolkata → 6
Mumbai → 7
Shillong → 8
Thiruvananthapuram → 9

 AQI_Bucket Encoding Mapping:
Good → 0
Moderate → 1
Poor → 2
Satisfactory → 3
Severe → 4
Very Poor → 5


,City,City_encoded,AQI_Bucket,AQI_Bucket_encoded
0,Delhi,2,Severe,4
4190,Mumbai,7,Satisfactory,3
4191,Mumbai,7,Satisfactory,3
1,Delhi,2,Severe,4
2,Delhi,2,Moderate,1


In [22]:
all_cities

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,City_encoded,AQI_Bucket_encoded
0,Delhi,2015-01-01,313.22,607.98,69.16,36.39,110.59,33.85,15.20,9.250,41.680,14.36,24.86,9.84,472.0,Severe,2,4
4190,Mumbai,2015-01-01,51.30,104.33,10.40,23.22,27.38,18.18,0.00,10.535,35.805,0.00,0.00,0.00,122.0,Satisfactory,7,3
4191,Mumbai,2015-01-02,51.30,104.33,10.40,23.22,26.20,18.18,0.00,10.535,35.805,0.00,0.00,0.00,122.0,Satisfactory,7,3
1,Delhi,2015-01-02,186.18,269.55,62.09,32.87,88.14,31.83,9.54,6.650,29.970,10.55,20.09,4.29,454.0,Severe,2,4
2,Delhi,2015-01-03,87.18,131.90,25.73,30.31,47.95,69.55,10.61,2.650,19.710,3.91,10.23,1.99,143.0,Moderate,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6198,Mumbai,2020-07-01,3.84,11.28,27.64,5.57,33.21,23.60,0.37,13.180,6.680,0.78,4.46,0.14,53.0,Satisfactory,7,3
2008,Delhi,2020-07-01,54.01,128.66,6.33,21.05,16.81,29.06,0.97,11.150,29.730,2.03,23.57,0.65,101.0,Moderate,2,1
3687,Gurugram,2020-07-01,61.64,174.08,5.99,10.58,12.37,2.91,1.11,7.570,44.140,2.67,4.46,4.73,157.0,Moderate,3,1
7012,Kolkata,2020-07-01,10.53,31.58,6.25,10.01,16.29,12.87,0.30,4.320,15.520,1.82,10.25,0.18,45.0,Good,6,0


In [23]:
if 'Unnamed: 0' in all_cities.columns:
    all_cities = all_cities.drop(columns=['Unnamed: 0'])
    print("Dropped 'Unnamed: 0' column.")
else:
    print("'Unnamed: 0' column not found in all_cities.")

print("\nUpdated DataFrame head:")
print(all_cities.head())
print("\nUpdated DataFrame columns:")
print(all_cities.columns)


'Unnamed: 0' column not found in all_cities.

Updated DataFrame head:
        City       Date   PM2.5    PM10     NO    NO2     NOx    NH3     CO  \
0      Delhi 2015-01-01  313.22  607.98  69.16  36.39  110.59  33.85  15.20   
4190  Mumbai 2015-01-01   51.30  104.33  10.40  23.22   27.38  18.18   0.00   
4191  Mumbai 2015-01-02   51.30  104.33  10.40  23.22   26.20  18.18   0.00   
1      Delhi 2015-01-02  186.18  269.55  62.09  32.87   88.14  31.83   9.54   
2      Delhi 2015-01-03   87.18  131.90  25.73  30.31   47.95  69.55  10.61   

         SO2      O3  Benzene  Toluene  Xylene    AQI    AQI_Bucket  \
0      9.250  41.680    14.36    24.86    9.84  472.0        Severe   
4190  10.535  35.805     0.00     0.00    0.00  122.0  Satisfactory   
4191  10.535  35.805     0.00     0.00    0.00  122.0  Satisfactory   
1      6.650  29.970    10.55    20.09    4.29  454.0        Severe   
2      2.650  19.710     3.91    10.23    1.99  143.0      Moderate   

      City_encoded  AQI_Buck

Train-Test Split

In [24]:
from sklearn.model_selection import train_test_split

# Drop non-numeric and irrelevant columns
X = all_cities[['PM2.5','PM10','NO','NO2','NOx','NH3',
        'CO','SO2','O3','Benzene','Toluene','Xylene','AQI']]


y = all_cities['AQI_Bucket_encoded']  # Target variable


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 20% test data
    random_state=42,    # reproducible split
    stratify=y          # ensures proportional target distribution
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Test target shape:", y_test.shape)

Training set shape: (8631, 13)
Test set shape: (2158, 13)
Training target shape: (8631,)
Test target shape: (2158,)


In [25]:
X_train

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
2588,62.93,104.33,5.95,10.10,28.20,18.18,0.49,3.650,20.320,0.02,1.91,0.14,228.0
4397,51.30,104.33,10.40,23.22,19.68,18.18,0.00,10.535,35.805,0.00,0.00,0.00,122.0
2086,51.30,104.33,9.17,15.92,11.05,18.18,1.57,11.110,18.560,1.22,4.46,0.14,122.0
5309,51.30,104.33,10.40,23.22,90.98,18.18,0.10,10.535,35.805,0.00,0.00,0.00,122.0
6866,78.36,161.52,30.92,56.10,87.99,25.32,0.81,13.530,38.610,20.47,25.77,2.63,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,130.13,409.70,43.34,103.25,32.82,54.57,0.54,30.820,56.860,2.39,14.76,0.14,357.0
8764,14.20,60.84,1.96,8.21,5.96,18.18,0.82,4.470,17.670,1.22,4.46,0.14,122.0
1050,191.57,278.95,38.61,53.86,82.34,50.06,1.55,22.890,62.010,4.06,12.03,0.12,357.0
2289,26.00,104.33,10.40,23.22,28.20,18.18,0.53,1.450,29.540,1.22,4.46,0.14,55.0


KNN Model,Random Forest Model,XGBoost Model

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

#  KNN CLASSIFIER


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)

print(" KNN Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\n Classification Report:\n", classification_report(y_test, y_pred_knn))



 KNN Classifier
Accuracy: 0.9388322520852641

 Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91       141
           1       0.93      0.94      0.93       544
           2       0.92      0.94      0.93       253
           3       0.96      0.96      0.96       946
           4       0.95      0.85      0.90        71
           5       0.91      0.93      0.92       203

    accuracy                           0.94      2158
   macro avg       0.93      0.92      0.92      2158
weighted avg       0.94      0.94      0.94      2158



In [27]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report

# Generate classification report for KNN
report_knn = classification_report(y_test, y_pred_knn, output_dict=True)

# Convert report to DataFrame for plotting, excluding 'accuracy', 'macro avg', 'weighted avg'
report_df_knn = pd.DataFrame(report_knn).transpose()
report_df_knn = report_df_knn.drop(columns=['support'])
report_df_knn = report_df_knn.drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')

# Add an 'AQI_Bucket' column for proper labeling
# Map encoded labels back to original AQI_Bucket names for better readability
report_df_knn['AQI_Bucket_encoded'] = report_df_knn.index.astype(int)

# Create a mapping from encoded value to original label
label_mapping = {code: bucket for bucket, code in zip(le_bucket.classes_, le_bucket.transform(le_bucket.classes_))}
report_df_knn['AQI_Bucket'] = report_df_knn['AQI_Bucket_encoded'].map(label_mapping)

# Melt the DataFrame for grouped bar chart
report_melted_knn = report_df_knn.melt(id_vars=['AQI_Bucket', 'AQI_Bucket_encoded'], var_name='Metric', value_name='Score')

# Define a specific order for AQI_Bucket for better visualization
aqi_bucket_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
report_melted_knn['AQI_Bucket'] = pd.Categorical(report_melted_knn['AQI_Bucket'], categories=aqi_bucket_order, ordered=True)
report_melted_knn = report_melted_knn.sort_values('AQI_Bucket')

# Create the grouped bar chart
fig = px.bar(report_melted_knn,
             x='AQI_Bucket',
             y='Score',
             color='Metric',
             barmode='group',
             title='KNN Model Performance: Precision, Recall, and F1-Score per AQI Bucket',
             labels={'AQI_Bucket': 'Air Quality Bucket', 'Score': 'Score'},
             hover_data={'Metric': True, 'Score': ':.2f'})

fig.update_layout(
    xaxis_title='Air Quality Bucket',
    yaxis_title='Score',
    yaxis_range=[0, 1.05] # Set y-axis range to 0-1 for scores
)

fig.show()


In [28]:
# RANDOM FOREST CLASSIFIER


rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("\n Random Forest Classifier ")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))





 Random Forest Classifier 
Accuracy: 0.9944392956441149

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       141
           1       0.98      1.00      0.99       544
           2       1.00      1.00      1.00       253
           3       1.00      0.99      0.99       946
           4       1.00      0.99      0.99        71
           5       1.00      1.00      1.00       203

    accuracy                           0.99      2158
   macro avg       1.00      1.00      1.00      2158
weighted avg       0.99      0.99      0.99      2158



In [29]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report

# Generate classification report for Random Forest
report_rf = classification_report(y_test, y_pred_rf, output_dict=True)

# Convert report to DataFrame for plotting, excluding 'accuracy', 'macro avg', 'weighted avg'
report_df_rf = pd.DataFrame(report_rf).transpose()
report_df_rf = report_df_rf.drop(columns=['support'])
report_df_rf = report_df_rf.drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')

# Add an 'AQI_Bucket' column for proper labeling
# Map encoded labels back to original AQI_Bucket names for better readability
report_df_rf['AQI_Bucket_encoded'] = report_df_rf.index.astype(int)

# Create a mapping from encoded value to original label (re-using label_mapping from previous cell)
# label_mapping = {code: bucket for bucket, code in zip(le_bucket.classes_, le_bucket.transform(le_bucket.classes_))}
report_df_rf['AQI_Bucket'] = report_df_rf['AQI_Bucket_encoded'].map(label_mapping)

# Melt the DataFrame for grouped bar chart
report_melted_rf = report_df_rf.melt(id_vars=['AQI_Bucket', 'AQI_Bucket_encoded'], var_name='Metric', value_name='Score')

# Define a specific order for AQI_Bucket for better visualization (re-using aqi_bucket_order)
# aqi_bucket_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
report_melted_rf['AQI_Bucket'] = pd.Categorical(report_melted_rf['AQI_Bucket'], categories=aqi_bucket_order, ordered=True)
report_melted_rf = report_melted_rf.sort_values('AQI_Bucket')

# Create the grouped bar chart
fig = px.bar(report_melted_rf,
             x='AQI_Bucket',
             y='Score',
             color='Metric',
             barmode='group',
             title='Random Forest Model Performance: Precision, Recall, and F1-Score per AQI Bucket',
             labels={'AQI_Bucket': 'Air Quality Bucket', 'Score': 'Score'},
             hover_data={'Metric': True, 'Score': ':.2f'})

fig.update_layout(
    xaxis_title='Air Quality Bucket',
    yaxis_title='Score',
    yaxis_range=[0, 1.05] # Set y-axis range to 0-1 for scores
)

fig.show()

In [30]:
# XGBOOST CLASSIFIER


xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='mlogloss'
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

print("\nXGBoost Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))


XGBoost Classifier
Accuracy: 0.9939759036144579

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       141
           1       0.99      0.99      0.99       544
           2       0.99      0.98      0.98       253
           3       1.00      1.00      1.00       946
           4       1.00      0.99      0.99        71
           5       0.99      0.99      0.99       203

    accuracy                           0.99      2158
   macro avg       0.99      0.99      0.99      2158
weighted avg       0.99      0.99      0.99      2158



In [31]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report

# Generate classification report for XGBoost
report_xgb = classification_report(y_test, y_pred_xgb, output_dict=True)

# Convert report to DataFrame for plotting, excluding 'accuracy', 'macro avg', 'weighted avg'
report_df_xgb = pd.DataFrame(report_xgb).transpose()
report_df_xgb = report_df_xgb.drop(columns=['support'])
report_df_xgb = report_df_xgb.drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')

# Add an 'AQI_Bucket' column for proper labeling
# Map encoded labels back to original AQI_Bucket names for better readability
report_df_xgb['AQI_Bucket_encoded'] = report_df_xgb.index.astype(int)

# Create a mapping from encoded value to original label (re-using label_mapping from previous cells)
# label_mapping = {code: bucket for bucket, code in zip(le_bucket.classes_, le_bucket.transform(le_bucket.classes_))}
report_df_xgb['AQI_Bucket'] = report_df_xgb['AQI_Bucket_encoded'].map(label_mapping)

# Melt the DataFrame for grouped bar chart
report_melted_xgb = report_df_xgb.melt(id_vars=['AQI_Bucket', 'AQI_Bucket_encoded'], var_name='Metric', value_name='Score')

# Define a specific order for AQI_Bucket for better visualization (re-using aqi_bucket_order)
# aqi_bucket_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
report_melted_xgb['AQI_Bucket'] = pd.Categorical(report_melted_xgb['AQI_Bucket'], categories=aqi_bucket_order, ordered=True)
report_melted_xgb = report_melted_xgb.sort_values('AQI_Bucket')

# Create the grouped bar chart
fig = px.bar(report_melted_xgb,
             x='AQI_Bucket',
             y='Score',
             color='Metric',
             barmode='group',
             title='XGBoost Model Performance: Precision, Recall, and F1-Score per AQI Bucket',
             labels={'AQI_Bucket': 'Air Quality Bucket', 'Score': 'Score'},
             hover_data={'Metric': True, 'Score': ':.2f'})

fig.update_layout(
    xaxis_title='Air Quality Bucket',
    yaxis_title='Score',
    yaxis_range=[0, 1.05] # Set y-axis range to 0-1 for scores
)

fig.show()

Model Evaluation(Accuracy & Precision Comparison Plots)

In [32]:
from sklearn.metrics import accuracy_score, classification_report, precision_score

# Calculate Accuracy & Precision for each model
accuracy_scores = {
    'KNN': accuracy_score(y_test, y_pred_knn),
    'Random Forest': accuracy_score(y_test, y_pred_rf),
    'XGBoost': accuracy_score(y_test, y_pred_xgb)
}

precision_scores = {
    'KNN': precision_score(y_test, y_pred_knn, average='weighted', zero_division=0),
    'Random Forest': precision_score(y_test, y_pred_rf, average='weighted', zero_division=0),
    'XGBoost': precision_score(y_test, y_pred_xgb, average='weighted', zero_division=0)
}

# Convert to DataFrame for Plotly
acc_df = pd.DataFrame(list(accuracy_scores.items()), columns=['Model', 'Accuracy'])
prec_df = pd.DataFrame(list(precision_scores.items()), columns=['Model', 'Precision'])

# Plot Accuracy using Plotly
fig_acc = px.bar(acc_df,
                 x='Model',
                 y='Accuracy',
                 color='Model', # Color each bar by model
                 title='Model Accuracy Comparison',
                 labels={'Accuracy': 'Accuracy Score'},
                 hover_data={'Model': True, 'Accuracy': ':.4f'})
fig_acc.update_layout(yaxis_range=[0, 1])
fig_acc.show()

# Plot Precision using Plotly
fig_prec = px.bar(prec_df,
                  x='Model',
                  y='Precision',
                  color='Model', # Color each bar by model
                  title='Model Precision Comparison',
                  labels={'Precision': 'Precision Score (Weighted)'},
                  hover_data={'Model': True, 'Precision': ':.4f'})
fig_prec.update_layout(yaxis_range=[0, 1])
fig_prec.show()

In [34]:
import joblib

# Save KNN Model
joblib.dump(knn, 'knn_model.pkl')
print("KNN model saved as knn_model.pkl")

# Save Random Forest Model
joblib.dump(rf, 'rf_model.pkl')
print("Random Forest model saved as rf_model.pkl")

# Save XGBoost Model
joblib.dump(xgb, 'xgb_model.pkl')
print("XGBoost model saved as xgb_model.pkl")

# Save the LabelEncoder for AQI_Bucket
joblib.dump(le_bucket, 'label_encoder.pkl')
print("Label encoder saved as label_encoder.pkl")

# Optionally, save the all_cities DataFrame if needed for the Streamlit app setup
# Since the Streamlit app reads from a CSV, we'll save the processed DataFrame as a CSV
all_cities.to_csv('all_cities.csv', index=False)
print("Processed all_cities DataFrame saved as all_cities.csv")

KNN model saved as knn_model.pkl
Random Forest model saved as rf_model.pkl
XGBoost model saved as xgb_model.pkl
Label encoder saved as label_encoder.pkl
Processed all_cities DataFrame saved as all_cities.csv


In [35]:
import joblib

knn = joblib.load("knn_model.pkl")
print("KNN expects features:", knn.n_features_in_)

rf = joblib.load("rf_model.pkl")
print("RF expects features:", rf.n_features_in_)

xgb = joblib.load("xgb_model.pkl")
print("XGB expects features:", xgb.n_features_in_)


KNN expects features: 13
RF expects features: 13
XGB expects features: 13


In [36]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install pyngrok

In [37]:
! pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.9 MB/s eta 0:00:00


In [38]:
!ngrok config add-authtoken 36e2mqK45xcOzPjDOHcQHy64QDl_7m2kcBk92naX8BSHzo7Sy

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [39]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import joblib

# ---------------------------------------------
# PAGE SETUP
# ---------------------------------------------
st.set_page_config(
    page_title="AQI Dashboard",
    page_icon="🌫️",
    layout="wide"
)

# ---------------------------------------------
# LOAD DATA & MODELS
# ---------------------------------------------
@st.cache_data
def load_data():
    df = pd.read_csv("all_cities.csv")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    return df

@st.cache_resource
def load_models():
    knn = joblib.load("knn_model.pkl")
    rf = joblib.load("rf_model.pkl")
    xgb = joblib.load("xgb_model.pkl")
    encoder = joblib.load("label_encoder.pkl")
    return knn, rf, xgb, encoder

df = load_data()
knn, rf, xgb, encoder = load_models()

# ---------------------------------------------
# SIDEBAR NAVIGATION
# ---------------------------------------------
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["📊 EDA Dashboard", "🤖 Model Prediction"])


# =========================================================
#              PAGE 1: EDA DASHBOARD
# =========================================================
if page == "📊 EDA Dashboard":

    st.title("📊 Exploratory Data Analysis (EDA)")
    st.markdown("Interactive visual analytics for Air Quality Dataset")

    st.subheader("Dataset Preview")
    st.write(df.head())

    # ----------------- City Count -----------------
    st.subheader("City Data Count")
    city_counts = df["City"].value_counts()
    st.bar_chart(city_counts)

    # ----------------- AQI Bucket Pie Chart -----------------
    st.subheader("AQI Bucket Distribution")
    bucket_counts = df["AQI_Bucket"].value_counts()

    fig = px.pie(
        names=bucket_counts.index,
        values=bucket_counts.values,
        title="AQI Category Distribution",
        hole=0.4
    )
    st.plotly_chart(fig, use_container_width=True)

    # ----------------- AQI Time Series -----------------
    st.subheader("AQI Time Series Trend")
    fig = px.line(
        df,
        x="Date",
        y="AQI",
        color="City",
        title="AQI Trend Over Time"
    )
    st.plotly_chart(fig, use_container_width=True)

    # ----------------- Pollutant Boxplot -----------------
    pollutants = ['PM2.5', 'PM10', 'NO2', 'O3']
    chosen_pollutant = st.selectbox("Select Pollutant for Boxplot", pollutants)

    fig = px.box(
        df,
        x="City",
        y=chosen_pollutant,
        color="City",
        title=f"{chosen_pollutant} Distribution Across Cities"
    )
    st.plotly_chart(fig, use_container_width=True)



# =========================================================
#              PAGE 2: MODEL PREDICTION
# =========================================================
else:

    st.title("🤖 AQI Bucket Prediction System")

    st.markdown("""
    Enter pollutant levels to predict the **AQI Category**
    Models available: **KNN, Random Forest, XGBoost**
    """)

    # ---- EXACT FEATURES USED DURING TRAINING ----
    feature_names = [
        'PM2.5','PM10','NO','NO2','NOx','NH3',
        'CO','SO2','O3','Benzene','Toluene','Xylene','AQI'
    ]

    inputs = {}

    st.subheader("Enter Pollutant Values")
    cols = st.columns(3)

    # 3-column input layout
    for i, feat in enumerate(feature_names):
        with cols[i % 3]:
            inputs[feat] = st.number_input(feat, min_value=0.0, step=0.1)

    # Select Model
    model_choice = st.radio(
        "Choose a Model",
        ["KNN", "Random Forest", "XGBoost"],
        horizontal=True
    )

    # Predict Button
    if st.button("Predict AQI Bucket"):

        values = np.array(list(inputs.values())).reshape(1, -1)

        if model_choice == "KNN":
            pred = knn.predict(values)
        elif model_choice == "Random Forest":
            pred = rf.predict(values)
        else:
            pred = xgb.predict(values)

        result = encoder.inverse_transform(pred)[0]

        st.success(f"### 🌫️ Predicted AQI Bucket: **{result}**")
        st.info("Prediction completed successfully!")


Writing app.py


In [40]:
!pkill -f ngrok

In [41]:
from pyngrok import ngrok
import subprocess

# Start Streamlit app
subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Open ngrok tunnel
public_url = ngrok.connect(addr='8501')
print(f"Access the Streamlit app here: {public_url}")

Access the Streamlit app here: NgrokTunnel: "https://nonmenacing-kelli-unplundered.ngrok-free.dev" -> "http://localhost:8501"
